In [1]:
from datetime import datetime
import requests, json, os
from typing import Dict, Annotated, Optional, List, Union
from dotenv import load_dotenv
load_dotenv()

from services.twilio import client
from services.cache import get_agent_metadata, get_all_agents
from services.chat.chat import similarity_search, get_embedding
from services.db.supabase_services import supabase_client
supabase = supabase_client()

from app.core.config import settings

from composio import Composio

# Initialize the Composio client
client = Composio()

In [ ]:
""" Get all connected apps and accounts """
""" store connection ID to each user_id in supabase """


# Get the list of integrations (connected apps)
integrations = client.integrations.get()
print("Connected Apps:")
for integration in integrations:
    print(f"• App: {integration.appName}")
    print(f"  ID: {integration.id}")

# Get the list of connected accounts (existing connections)
connected_accounts = client.connected_accounts.get()
print("\nExisting Connections:")
for account in connected_accounts:
    print(f"• ID: {account.id}")
    print(f"  App: {account.appName}")
    print(f"  Status: {account.status}")


In [ ]:
from services.composio import get_calendar_slots

get_calendar_slots("user_2mmXezcGmjZCf88gT2v2waCBsXv")

In [75]:
from composio import Composio

# Initialize the Composio client
composio_client = Composio()

# Create a new entity with a custom ID
new_entity = composio_client.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXv")


In [ ]:
""" checks if entity already exists """

existing_entity = composio_client.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXv")
existing_connections = existing_entity.get_connections()

if existing_connections:
    print("Entity already exists with connections:", existing_connections)
else:
    print("Entity is newly created or has no connections")


# # Check for connected accounts for the new entity
# try:
#     connected_account = new_entity.get_connection(app="outlook")
#     print(f"Connected Outlook account: {connected_account}")
# except Exception as e:
#     print("No connected Slack account found")

# # Perform actions using the new entity
# actions = new_entity.get_tools(app_name="github")

existing_connections

### Calendar Management

In [ ]:
""" this function will be triggered in endpoint /composion/new_connection """
""" then to store the connection ID to the user_id in supabase """
""" frontend to have button to "generate auth link", response from this endpoint will be the redirect url """
### entity ID will be created for each user. 
""" {"app": "outlook", "connectedAccountId" : "xxx", "client_id" : "xxx" } """

"""Auth"""
from composio import ComposioToolSet, App
from composio import Composio
from composio_openai import ComposioToolSet, Action
from openai import OpenAI

# Initialize the Composio client
composio_client = Composio()

# # Create a new entity with a custom ID
# new_entity = composio_client.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXv")

toolset = ComposioToolSet()
entity = toolset.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXvww")

# request = entity.initiate_connection(App.OUTLOOK)
# ### background task save request.connectedAccountId to integrations table in supabase 
# ### then return the below redirect url to frontend
# print(f"Open this URL to authenticate: {request.redirectUrl}")


In [ ]:
tools = toolset.get_tools(actions=[Action.OUTLOOK_LIST_EVENTS])

tools

In [ ]:
tools = toolset.get_tools(apps=[App.OUTLOOK])

tools

In [62]:
openai_client = OpenAI()

today = datetime.now().strftime("%Y-%m-%d")
task = f"get calendar slots for next week, today is {today}"

# Update the tools to include search capability
tools = toolset.get_tools(actions=[
    Action.OUTLOOK_OUTLOOK_UPDATE_CALENDAR_EVENT])


# First, let's search for your database
search_response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
    ],
)
search_result = toolset.handle_tool_calls(search_response)
search_result


In [ ]:
search_result

### Shopify

In [ ]:
"""Auth"""
""" shopify not working :("""

from composio import ComposioToolSet, App

toolset = ComposioToolSet()
entity = toolset.get_entity()
request = entity.initiate_connection(App.SHOPIFY)

print(f"Open this URL to authenticate: {request.redirectUrl}")


### Notion

In [ ]:
"""Auth"""

from composio import ComposioToolSet, App

toolset = ComposioToolSet(entity_id="Jessica")
entity = toolset.get_entity()
request = entity.initiate_connection(App.NOTION)

print(f"Open this URL to authenticate: {request.redirectUrl}")


In [ ]:
from composio_openai import ComposioToolSet, Action
from openai import OpenAI

openai_client = OpenAI()
composio_toolset = ComposioToolSet(entity_id="Jessica")

In [ ]:
today = datetime.now().strftime("%Y-%m-%d")
task = "get the DB id for DevOps DB"

# Update the tools to include search capability
tools = composio_toolset.get_tools(actions=[
    Action.NOTION_SEARCH_NOTION_PAGE])

# First, let's search for your database
search_response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
    ],
)
search_result = composio_toolset.handle_tool_calls(search_response)

db_id = search_result[0]['data']['response_data']['results'][0]['id']
print("DB ID:", db_id)

task = f"get the page to do's in DB with id {db_id}"

# Update the tools to include search capability
tools = composio_toolset.get_tools(actions=[
    Action.NOTION_QUERY_DATABASE])

search_response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
    ],
)
search_result = composio_toolset.handle_tool_calls(search_response)



In [ ]:
print(search_result)


### Google Calendar

In [2]:
from composio_openai import ComposioToolSet, Action
from openai import OpenAI

openai_client = OpenAI()
composio_toolset = ComposioToolSet(entity_id="user_2mmXezcGmjZCf88gT2v2waCBsXv")


GOOGLECALENDAR_CREATE_EVENT: 'Action'
GOOGLECALENDAR_DELETE_EVENT: 'Action'
GOOGLECALENDAR_DUPLICATE_CALENDAR: 'Action'
GOOGLECALENDAR_FIND_EVENT: 'Action'
GOOGLECALENDAR_FIND_FREE_SLOTS: 'Action'
GOOGLECALENDAR_GET_CALENDAR: 'Action'
GOOGLECALENDAR_GET_CURRENT_DATE_TIME: 'Action'
GOOGLECALENDAR_LIST_CALENDARS: 'Action'
GOOGLECALENDAR_PATCH_CALENDAR: 'Action'
GOOGLECALENDAR_QUICK_ADD: 'Action'
GOOGLECALENDAR_REMOVE_ATTENDEE: 'Action'
GOOGLECALENDAR_UPDATE_EVENT: 'Action'

# tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_LIST_CALENDARS])
# task = "find free slots for next week"


# tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_GET_CALENDAR])
# task = "fetch calendar for id michael@flowon.ai"

tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_FIND_FREE_SLOTS])
today = datetime.now().strftime("%Y-%m-%d")

task = f"Search for free slots for the next 14 days for id michael@flowon.ai. Today's date is {today}."

response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": f"You are a helpful assistant. Today's date is {today}."},
        {"role": "user", "content": task},
    ],
)
result = composio_toolset.handle_tool_calls(response)

[2024-11-25 20:11:49,245][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this
[2024-11-25 20:11:55,485][INFO] Executing `GOOGLECALENDAR_FIND_FREE_SLOTS` with params={'time_min': '2024,11,25,00,00,00', 'time_max': '2024,12,09,00,00,00', 'items': ['michael@flowon.ai'], 'timezone': 'UTC'} and metadata={} connected_account_id=None
[2024-11-25 20:11:57,094][INFO] Got response={'successfull': True, 'data': {'response_data': {'kind': 'calendar#freeBusy', 'timeMin': '2024-11-25T00:00:00.000Z', 'timeMax': '2024-12-09T00:00:00.000Z', 'calendars': {'michael@flowon.ai': {'busy': [{'start': '2024-11-25T12:30:00Z', 'end': '...


In [6]:
from services.composio import get_calendar_slots

await get_calendar_slots("user_2mmXezcGmjZCf88gT2v2waCBsXv","outlook")

[2024-11-25 20:25:32,156][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this


[2024-11-25 20:25:35,038][INFO] Executing `GOOGLECALENDAR_FIND_FREE_SLOTS` with params={'time_min': '2024,11,25,20,25,00', 'time_max': '2024,12,09,20,25,00', 'items': ['michael@flowon.ai'], 'timezone': 'UTC'} and metadata={} connected_account_id=None
[2024-11-25 20:25:36,694][INFO] Got response={'successfull': True, 'data': {'response_data': {'kind': 'calendar#freeBusy', 'timeMin': '2024-11-25T20:25:00.000Z', 'timeMax': '2024-12-09T20:25:00.000Z', 'calendars': {'michael@flowon.ai': {'busy': [{'start': '2024-11-26T14:00:00Z', 'end': '...


[{'start': '2024-11-26T09:00:00+00:00', 'end': '2024-11-26T09:30:00+00:00'},
 {'start': '2024-11-26T09:30:00+00:00', 'end': '2024-11-26T10:00:00+00:00'},
 {'start': '2024-11-26T10:00:00+00:00', 'end': '2024-11-26T10:30:00+00:00'},
 {'start': '2024-11-26T10:30:00+00:00', 'end': '2024-11-26T11:00:00+00:00'},
 {'start': '2024-11-26T11:00:00+00:00', 'end': '2024-11-26T11:30:00+00:00'},
 {'start': '2024-11-26T11:30:00+00:00', 'end': '2024-11-26T12:00:00+00:00'},
 {'start': '2024-11-26T12:00:00+00:00', 'end': '2024-11-26T12:30:00+00:00'},
 {'start': '2024-11-26T12:30:00+00:00', 'end': '2024-11-26T13:00:00+00:00'}]

In [3]:
from datetime import datetime, timedelta
import pytz

def find_free_slots(calendar_data, num_slots=8, duration_minutes=30):
    busy_slots = calendar_data
    
    # Convert to datetime objects and sort
    busy_slots = [(datetime.fromisoformat(slot['start'].replace('Z', '+00:00')),
                   datetime.fromisoformat(slot['end'].replace('Z', '+00:00')))
                  for slot in busy_slots]
    busy_slots.sort()
    
    # Start from current time
    current_time = datetime.now(pytz.UTC)
    # Round up to the next half hour
    current_time = current_time + timedelta(minutes=(30 - current_time.minute % 30))
    
    free_slots = []
    # Look ahead for 5 business days
    days_to_check = 5
    current_day = current_time.date()
    
    while len(free_slots) < num_slots and days_to_check > 0:
        # Set day start to 9 AM
        day_start = datetime.combine(current_day, datetime.min.time().replace(hour=9), tzinfo=pytz.UTC)
        day_end = datetime.combine(current_day, datetime.min.time().replace(hour=17), tzinfo=pytz.UTC)
        
        # If it's today, start from current time
        if current_day == current_time.date():
            day_start = max(day_start, current_time)
        
        # Get busy slots for this day
        day_busy_slots = [slot for slot in busy_slots 
                         if slot[0].date() == current_day]
        
        slot_start = day_start
        
        # If there are busy slots on this day
        if day_busy_slots:
            for busy_start, busy_end in day_busy_slots:
                # Check for free time before busy slot
                while slot_start + timedelta(minutes=duration_minutes) <= busy_start and slot_start + timedelta(minutes=duration_minutes) <= day_end:
                    free_slots.append({
                        'start': slot_start.isoformat(),
                        'end': (slot_start + timedelta(minutes=duration_minutes)).isoformat()
                    })
                    slot_start += timedelta(minutes=duration_minutes)
                    if len(free_slots) >= num_slots:
                        return free_slots
                
                # Move start time to after the busy slot
                slot_start = max(slot_start, busy_end)
        
        # Check remaining time until end of business day
        while slot_start + timedelta(minutes=duration_minutes) <= day_end:
            free_slots.append({
                'start': slot_start.isoformat(),
                'end': (slot_start + timedelta(minutes=duration_minutes)).isoformat()
            })
            slot_start += timedelta(minutes=duration_minutes)
            if len(free_slots) >= num_slots:
                return free_slots
        
        # Move to next day
        current_day += timedelta(days=1)
        days_to_check -= 1
    
    return free_slots

# Example usage:
calendar_data = result[0]['data']['response_data']['calendars']['michael@flowon.ai']['busy']

free_slots = find_free_slots(calendar_data)
for slot in free_slots:
    print(f"Available: {slot['start']} - {slot['end']}")

Available: 2024-11-26T09:00:00+00:00 - 2024-11-26T09:30:00+00:00
Available: 2024-11-26T09:30:00+00:00 - 2024-11-26T10:00:00+00:00
Available: 2024-11-26T10:00:00+00:00 - 2024-11-26T10:30:00+00:00
Available: 2024-11-26T10:30:00+00:00 - 2024-11-26T11:00:00+00:00
Available: 2024-11-26T11:00:00+00:00 - 2024-11-26T11:30:00+00:00
Available: 2024-11-26T11:30:00+00:00 - 2024-11-26T12:00:00+00:00
Available: 2024-11-26T12:00:00+00:00 - 2024-11-26T12:30:00+00:00
Available: 2024-11-26T12:30:00+00:00 - 2024-11-26T13:00:00+00:00


In [ ]:
def parse_calendar_slots(result):
    # Extract the calendar data
    calendar_data = result[0]['data']['response_data']['calendars']
    email = list(calendar_data.keys())[0]
    busy_slots = calendar_data[email]['busy']
    
    # Convert busy slots to free slots
    time_min = result[0]['data']['response_data']['timeMin']
    time_max = result[0]['data']['response_data']['timeMax']
    
    # Convert to datetime objects
    start_day = datetime.strptime(time_min, '%Y-%m-%dT%H:%M:%S.%fZ')
    end_day = datetime.strptime(time_max, '%Y-%m-%dT%H:%M:%S.%fZ')
    
    # Create a list of busy periods
    busy_periods = []
    for slot in busy_slots:
        start = datetime.strptime(slot['start'], '%Y-%m-%dT%H:%M:%SZ')
        end = datetime.strptime(slot['end'], '%Y-%m-%dT%H:%M:%SZ')
        busy_periods.append((start, end))
    
    # Format the schedule in a human-readable way
    schedule_summary = f"Schedule for {email} on {start_day.strftime('%Y-%m-%d')}:\n\n"
    schedule_summary += "Busy periods:\n"
    for start, end in busy_periods:
        schedule_summary += f"- {start.strftime('%H:%M')} to {end.strftime('%H:%M')}\n"
    
    return schedule_summary

# Use the function
schedule = parse_calendar_slots(result)
print(schedule)

In [ ]:

tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_CREATE_EVENT])

task = "Create a 1 hour meeting event at 5:30PM tomorrow regarding OpenAI Integration"

today = datetime.now().strftime("%Y-%m-%d")

response = openai_client.chat.completions.create(
    model="gpt-4-turbo-preview",
    tools=tools,
    messages=[
        {"role": "system", "content": f"You are a helpful assistant. Today's date is {today}."},
        {"role": "user", "content": task},
    ],
)
result = composio_toolset.handle_tool_calls(response)
print(result)
